In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose 

# 1.x Save Video

In [23]:
cap = cv2.VideoCapture(0)

#비디오 속성 확인
height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps=cap.get(cv2.CAP_PROP_FPS) #초당 프레임수

#비디오 레코드 저장
videoWriter= cv2.VideoWriter('train_spine_1.avi',cv2.VideoWriter_fourcc('P','I','M','1'),fps,(int(width),int(height)))

while cap.isOpened():
    ret,frame = cap.read()
    
    try:
        cv2.imshow('Press',frame)
        videoWriter.write(frame)
        
    except Exception as e:
        break
        
    if cv2.waitKey(10) & 0xFF ==ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.

# 2. Capture Landmarks & Export to CSV

In [15]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [16]:
landmarks=['spine']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [17]:
# 데이터 저장할 csv 파일 형성
with open('coords_spine.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [18]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        
        if action=='good':
            keypoints = np.insert(keypoints,0,0)

        if action=='left':
            keypoints = np.insert(keypoints,0,1)
            
        if action=='right':
            keypoints = np.insert(keypoints,0,2)
            
        with open('coords_spine.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)

    except Exception as e:
        pass

In [24]:
# 영상을 통해 데이터셋 구축
cap =cv2.VideoCapture("train_spine_1.avi") #setting video capture device(number은 웹캠을 대표하는 숫자)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret,frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
      
        #Detect stuff and render
        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환

        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(155,117,166),thickness=2,circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(195,116,230),thickness=2,circle_radius=2))
        
        #키보드로 입력한 키와 동일한 아스키코드값
        k = cv2.waitKey(1)
        
        if k == 103: # good의 g
            print('good')
            export_landmark(results,'good')
            
        if k == 108: # left의 l
            print('left')
            export_landmark(results,'left')
            
        if k == 114: # right의 r
            print('right')
            export_landmark(results,'right')
            

        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.

good
good
good
good
good
left
left
left
left
left
good
good
good
good
good
right
right
right
right
right
good
good
good
good
good
left
left
left
left
left
good
good
good
good
good


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# 3. Train Custom Model Using Scikit Learn

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [19]:
df = pd.read_csv('coords_spine.csv')

In [20]:
df[df['spine']==1] #왼쪽 치우침

,spine,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
9,1,0.660415,0.502327,-0.549419,0.999956,0.682633,0.436963,-0.512078,0.999897,0.698794,...,0.328144,0.000004,0.590704,3.144917,0.169610,0.000093,0.435877,3.117650,-0.115816,0.000024
10,1,0.790604,0.570801,-0.655782,0.999946,0.811914,0.508568,-0.607615,0.999867,0.824076,...,0.314946,0.000037,0.641212,3.210237,0.215872,0.000465,0.472981,3.159673,-0.182655,0.000162
11,1,0.686590,0.530080,-0.673420,0.999963,0.709449,0.463909,-0.633155,0.999898,0.722252,...,0.473592,0.000025,0.588326,3.195791,0.271277,0.000348,0.419603,3.164313,0.006168,0.000121
16,1,0.665880,0.503905,-0.609042,0.999892,0.689702,0.443389,-0.569539,0.999745,0.701845,...,0.341734,0.000020,0.597039,3.130493,0.258903,0.000233,0.429165,3.108145,-0.100173,0.000066
21,1,0.788539,0.542069,-0.648995,0.999785,0.805868,0.477491,-0.603133,0.999520,0.816257,...,0.422982,0.000020,0.646095,3.134066,0.318761,0.000241,0.484307,3.103844,-0.051438,0.000066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,1,0.657888,0.668478,-0.625523,0.999925,0.673880,0.604335,-0.579291,0.999824,0.687032,...,0.602788,0.000015,0.680551,3.126434,0.177587,0.000073,0.531945,3.102200,0.151375,0.000044
278,1,0.602837,0.659116,-0.500433,0.999963,0.626958,0.595306,-0.462578,0.999893,0.641760,...,0.379937,0.000004,0.632853,3.122994,0.084553,0.000040,0.490242,3.098466,-0.039019,0.000022
286,1,0.713514,0.675523,-0.686126,0.999899,0.727625,0.613874,-0.642822,0.999785,0.741079,...,0.661895,0.000024,0.688997,3.115209,0.134220,0.000136,0.544093,3.089793,0.190426,0.000072
287,1,0.746520,0.684992,-0.653861,0.999605,0.759177,0.624782,-0.604130,0.999199,0.772037,...,0.644361,0.000059,0.728908,3.123887,0.137540,0.000292,0.580152,3.110631,0.166976,0.000162


In [60]:
df[df['spine']==2] #오른쪽 치우침

,shol,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
12,2.0,0.433727,0.505592,-0.560400,0.999930,0.453073,0.434054,-0.534789,0.999803,0.468615,...,0.290182,0.000018,0.579033,3.070248,-0.114390,0.000098,0.447308,3.064416,-0.143134,0.000046
13,2.0,0.353145,0.519404,-0.478921,0.999920,0.370915,0.447676,-0.482776,0.999789,0.385782,...,0.415291,0.000010,0.585151,3.075493,-0.032263,0.000055,0.438569,3.085943,-0.001289,0.000027
14,2.0,0.408024,0.492308,-0.614546,0.999920,0.431026,0.424512,-0.584016,0.999797,0.446245,...,0.513835,0.000007,0.583222,3.131412,0.176710,0.000037,0.441414,3.125983,0.073359,0.000018
18,2.0,0.421857,0.501464,-0.574367,0.999939,0.436615,0.423910,-0.561442,0.999819,0.451750,...,0.467530,0.000019,0.615349,3.057913,0.045524,0.000093,0.472512,3.068501,0.023508,0.000047
19,2.0,0.358268,0.516691,-0.506649,0.999867,0.371607,0.441273,-0.510856,0.999668,0.384747,...,0.498405,0.000011,0.591005,3.098252,-0.022578,0.000048,0.448067,3.108497,0.063330,0.000027
25,2.0,0.379135,0.498918,-0.556605,0.999937,0.394851,0.429372,-0.531065,0.999822,0.408835,...,0.360309,0.000015,0.576050,3.053942,-0.054516,0.000075,0.439296,3.055231,-0.085519,0.000039
26,2.0,0.400433,0.503466,-0.602360,0.999916,0.419821,0.432429,-0.557266,0.999760,0.435217,...,0.471918,0.000009,0.581320,3.072355,0.137366,0.000047,0.440685,3.069596,0.036909,0.000023
32,2.0,0.446678,0.478212,-0.844583,0.999935,0.465322,0.408916,-0.809462,0.999809,0.481683,...,0.351356,0.000017,0.585788,3.111898,-0.054769,0.000090,0.448216,3.100482,-0.106973,0.000049
33,2.0,0.359437,0.492350,-0.515323,0.999852,0.375991,0.421587,-0.521758,0.999582,0.390937,...,0.417126,0.000008,0.577365,3.005578,0.008688,0.000041,0.444373,3.006032,-0.008295,0.000022
40,2.0,0.302854,0.493907,-0.565102,0.999933,0.320400,0.420285,-0.562888,0.999771,0.334415,...,0.335508,0.000005,0.516063,3.062707,-0.053108,0.000040,0.385580,3.063695,-0.110307,0.000017


In [21]:
X = df.drop('spine',axis=1) #shol을 제외한 입력 feature만 남김.
y = df['spine']

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123) #어깨

In [63]:
y_test

10    1.0
13    2.0
30    1.0
48    2.0
18    2.0
0     0.0
40    2.0
12    2.0
29    1.0
8     0.0
21    1.0
46    2.0
11    1.0
41    2.0
5     0.0
Name: shol, dtype: float64

https://camo.githubusercontent.com/37bc1d0f546f2dd001798a0f66057c9c907e6b5efae95a068bcce8c852c2e234/68747470733a2f2f692e696d6775722e636f6d2f336a38425064632e706e67

In [23]:
df.drop(['x13'],axis=1,inplace=True)
df.drop(['y13'],axis=1,inplace=True)
df.drop(['z13'],axis=1,inplace=True)
df.drop(['v13'],axis=1,inplace=True)

df.drop(['x14'],axis=1,inplace=True)
df.drop(['y14'],axis=1,inplace=True)
df.drop(['z14'],axis=1,inplace=True)
df.drop(['v14'],axis=1,inplace=True)

df.drop(['x15'],axis=1,inplace=True)
df.drop(['y15'],axis=1,inplace=True)
df.drop(['z15'],axis=1,inplace=True)
df.drop(['v15'],axis=1,inplace=True)

df.drop(['x16'],axis=1,inplace=True)
df.drop(['y16'],axis=1,inplace=True)
df.drop(['z16'],axis=1,inplace=True)
df.drop(['v16'],axis=1,inplace=True)

df.drop(['x17'],axis=1,inplace=True)
df.drop(['y17'],axis=1,inplace=True)
df.drop(['z17'],axis=1,inplace=True)
df.drop(['v17'],axis=1,inplace=True)

df.drop(['x18'],axis=1,inplace=True)
df.drop(['y18'],axis=1,inplace=True)
df.drop(['z18'],axis=1,inplace=True)
df.drop(['v18'],axis=1,inplace=True)

df.drop(['x19'],axis=1,inplace=True)
df.drop(['y19'],axis=1,inplace=True)
df.drop(['z19'],axis=1,inplace=True)
df.drop(['v19'],axis=1,inplace=True)

df.drop(['x20'],axis=1,inplace=True)
df.drop(['y20'],axis=1,inplace=True)
df.drop(['z20'],axis=1,inplace=True)
df.drop(['v20'],axis=1,inplace=True)

df.drop(['x21'],axis=1,inplace=True)
df.drop(['y21'],axis=1,inplace=True)
df.drop(['z21'],axis=1,inplace=True)
df.drop(['v21'],axis=1,inplace=True)

df.drop(['x22'],axis=1,inplace=True)
df.drop(['y22'],axis=1,inplace=True)
df.drop(['z22'],axis=1,inplace=True)
df.drop(['v22'],axis=1,inplace=True)

df.drop(['x23'],axis=1,inplace=True)
df.drop(['y23'],axis=1,inplace=True)
df.drop(['z23'],axis=1,inplace=True)
df.drop(['v23'],axis=1,inplace=True)

df.drop(['x24'],axis=1,inplace=True)
df.drop(['y24'],axis=1,inplace=True)
df.drop(['z24'],axis=1,inplace=True)
df.drop(['v24'],axis=1,inplace=True)

df.drop(['x25'],axis=1,inplace=True)
df.drop(['y25'],axis=1,inplace=True)
df.drop(['z25'],axis=1,inplace=True)
df.drop(['v25'],axis=1,inplace=True)

df.drop(['x26'],axis=1,inplace=True)
df.drop(['y26'],axis=1,inplace=True)
df.drop(['z26'],axis=1,inplace=True)
df.drop(['v26'],axis=1,inplace=True)

df.drop(['x27'],axis=1,inplace=True)
df.drop(['y27'],axis=1,inplace=True)
df.drop(['z27'],axis=1,inplace=True)
df.drop(['v27'],axis=1,inplace=True)

df.drop(['x28'],axis=1,inplace=True)
df.drop(['y28'],axis=1,inplace=True)
df.drop(['z28'],axis=1,inplace=True)
df.drop(['v28'],axis=1,inplace=True)

df.drop(['x29'],axis=1,inplace=True)
df.drop(['y29'],axis=1,inplace=True)
df.drop(['z29'],axis=1,inplace=True)
df.drop(['v29'],axis=1,inplace=True)

df.drop(['x30'],axis=1,inplace=True)
df.drop(['y30'],axis=1,inplace=True)
df.drop(['z30'],axis=1,inplace=True)
df.drop(['v30'],axis=1,inplace=True)

df.drop(['x31'],axis=1,inplace=True)
df.drop(['y31'],axis=1,inplace=True)
df.drop(['z31'],axis=1,inplace=True)
df.drop(['v31'],axis=1,inplace=True)

df.drop(['x32'],axis=1,inplace=True)
df.drop(['y32'],axis=1,inplace=True)
df.drop(['z32'],axis=1,inplace=True)
df.drop(['v32'],axis=1,inplace=True)

df.drop(['x33'],axis=1,inplace=True)
df.drop(['y33'],axis=1,inplace=True)
df.drop(['z33'],axis=1,inplace=True)
df.drop(['v33'],axis=1,inplace=True)

In [24]:
df

,spine,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z10,v10,x11,y11,z11,v11,x12,y12,z12,v12
0,0,0.543709,0.524400,-0.569407,0.999989,0.567922,0.459572,-0.536507,0.999967,0.581814,...,-0.484794,0.999982,0.514014,0.596661,-0.480938,0.999988,0.733565,0.901072,-0.189265,0.999300
1,0,0.558027,0.499163,-0.688157,0.999974,0.579239,0.433892,-0.642779,0.999927,0.591282,...,-0.578711,0.999954,0.529008,0.574013,-0.595632,0.999972,0.731214,0.863917,-0.159800,0.999147
2,0,0.568836,0.497314,-0.663163,0.999976,0.588036,0.432810,-0.618399,0.999941,0.601490,...,-0.561716,0.999967,0.542423,0.570753,-0.568092,0.999969,0.734886,0.873584,-0.181027,0.999395
3,0,0.572842,0.500612,-0.553825,0.999985,0.591226,0.436276,-0.517266,0.999964,0.604136,...,-0.464990,0.999977,0.543537,0.572848,-0.466101,0.999980,0.742385,0.862153,-0.157847,0.999482
4,0,0.536347,0.485976,-0.519811,0.999985,0.557511,0.431650,-0.486413,0.999955,0.572071,...,-0.438862,0.999967,0.510053,0.560505,-0.443975,0.999980,0.719222,0.862548,-0.172011,0.999234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,2,0.382666,0.629277,-0.540843,0.999972,0.404483,0.567808,-0.505049,0.999922,0.416922,...,-0.457965,0.999948,0.353834,0.699415,-0.453556,0.999959,0.558648,0.965553,-0.172356,0.998900
294,2,0.482934,0.623939,-0.504504,0.999978,0.498852,0.564779,-0.467317,0.999945,0.511748,...,-0.425534,0.999961,0.462340,0.692858,-0.421950,0.999968,0.651060,0.968525,-0.145714,0.999134
295,0,0.558619,0.633654,-0.512488,0.999976,0.576540,0.569164,-0.478859,0.999944,0.589357,...,-0.433944,0.999960,0.536030,0.708812,-0.425959,0.999963,0.734001,0.976105,-0.153411,0.999168
296,0,0.548865,0.657911,-0.828266,0.999699,0.572691,0.594254,-0.774898,0.999524,0.586732,...,-0.695009,0.999558,0.522010,0.732256,-0.703386,0.999539,0.727077,0.980747,-0.178120,0.989111


In [25]:
df.mean()

spine    0.805369
x1       0.547935
y1       0.594241
z1      -0.621977
v1       0.999902
x2       0.567887
y2       0.530205
z2      -0.582550
v2       0.999776
x3       0.581526
y3       0.532104
z3      -0.582482
v3       0.999822
x4       0.594283
y4       0.534929
z4      -0.582533
v4       0.999753
x5       0.527216
y5       0.530088
z5      -0.587486
v5       0.999799
x6       0.513180
y6       0.531566
z6      -0.586965
v6       0.999849
x7       0.500142
y7       0.533656
z7      -0.587225
v7       0.999809
x8       0.613453
y8       0.573038
z8      -0.324079
v8       0.999757
x9       0.482957
y9       0.570523
z9      -0.332083
v9       0.999864
x10      0.573838
y10      0.667644
z10     -0.523733
v10      0.999852
x11      0.523392
y11      0.666745
z11     -0.526215
v11      0.999887
x12      0.720893
y12      0.929248
z12     -0.163308
v12      0.997523
dtype: float64

In [26]:
for val in range(1,12+1):
    print(df['v{}'.format(val)].mean())

0.9999024381744959
0.999776186738255
0.9998217816812078
0.999752922956376
0.9997992347516772
0.9998487064295299
0.9998093103221474
0.9997570564463089
0.9998640838087246
0.9998518566644291
0.9998867461979869
0.9975226579228191


## 3.2 Train Machine Learning Classification Model

In [27]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [33]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(),LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [34]:
fit_models={}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [35]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [36]:
fit_models['rf'].predict(X_test)

array([2, 1, 2, 0, 2, 2, 1, 1, 2, 1, 0, 0, 2, 2, 1, 0, 0, 2, 1, 0, 1, 2,
       2, 0, 1, 0, 0, 0, 0, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 0, 0, 2, 2, 0,
       0, 2, 0, 1, 2, 0, 1, 0, 1, 0, 2, 2, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       2, 0, 1, 1, 1, 1, 1, 2, 0, 1, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 2, 0,
       0, 0], dtype=int64)

In [38]:
y_test

72     2
54     1
82     2
276    0
94     2
      ..
102    1
159    2
204    0
255    0
127    0
Name: spine, Length: 90, dtype: int64

## 3.3 Evaluate and Serialize Model

In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [47]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test.values,yhat),
    precision_score(y_test.values,yhat,average=None),
    recall_score(y_test.values,yhat,average=None))

lr 0.9777777777777777 [0.97142857 0.96666667 1.        ] [0.97142857 0.96666667 1.        ]
rc 0.9222222222222223 [0.88888889 0.9        1.        ] [0.91428571 0.9        0.96      ]
rf 0.9777777777777777 [0.97142857 0.96666667 1.        ] [0.97142857 0.96666667 1.        ]
gb 0.9333333333333333 [0.91428571 0.93333333 0.96      ] [0.91428571 0.93333333 0.96      ]


In [48]:
yhat = fit_models['rf'].predict(X_test)

In [75]:
yhat[:10]

array([1., 2., 1., 2., 2., 0., 2., 2., 1., 0.])

In [76]:
y_test

10    1.0
13    2.0
30    1.0
48    2.0
18    2.0
0     0.0
40    2.0
12    2.0
29    1.0
8     0.0
21    1.0
46    2.0
11    1.0
41    2.0
5     0.0
Name: shol, dtype: float64

In [49]:
with open('Spine.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

# 4. Make Detections with Model

In [50]:
with open('Spine.pkl','rb') as f:
    model = pickle.load(f)

In [51]:
landmarks=['spine']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [54]:
#VISUALIZE DEGREE
cap =cv2.VideoCapture(0) #setting video capture device(number은 웹캠을 대표하는 숫자)

# Curl counter variables
counter = 0

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret, frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
      
        #Detect stuff and render
        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환
        
         #Extract landmarks
        try:  
            row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
            X = pd.DataFrame([row],columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            if body_language_class == 0 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'Good'
            elif current_stage == 'Good' and body_language_class ==1 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'left'
                counter +=1
            elif current_stage == 'Good' and body_language_class ==2 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'right'
                counter +=1
        
            #Setup status bow
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1) 


            #지금 상태
            cv2.putText(image,'CLASS',(15,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image, 'left' if body_language_class == 1 else 'right' if body_language_class == 2 else 'Good',(15,40)
                        ,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
            #Stage data 
            cv2.putText(image,'Count',(180,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
    
            cv2.putText(image,str(counter),(175,40),
                       cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            # Render detections
            #drawing 툴을 사용해서 감지된 자세포인트와 이들 간의 포인트를 연결해 보여준다.
            #mp_drawing.DrawingSpec은 관절부위와 bone 부분의 색깔, 굵기 등을 지정해준다.
            mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                     mp_drawing.DrawingSpec(color=(155,117,166),thickness=2,circle_radius=2),
                                     mp_drawing.DrawingSpec(color=(195,116,230),thickness=2,circle_radius=2))


        except: #error가 있으면 실행x
            pass
        
        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.